# Pruning

このチャプターでは、modelのpruningを行っていきます。

In [ ]:
%pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import os
import zipfile

### データセットの準備
評価で使用するため、再度Fashion-MNISTデータセットをロードして、
前処理も行なっておきます。

In [ ]:
(X_train_orig, y_train_orig), (X_test_orig, y_test_orig) = fashion_mnist.load_data()

## shapeを(batch_size, rows, cols, channels)にexpandする
X_train = np.expand_dims(X_train_orig, -1)
X_test = np.expand_dims(X_test_orig, -1)

print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)

## グレースケールの 0-255 の値を 正規化して 0-1 の浮動小数にする
X_train = X_train / 255.0
X_test = X_test / 255.0

## one hot vectorにする
y_train = tf.keras.utils.to_categorical(y_train_orig, 10)
y_test = tf.keras.utils.to_categorical(y_test_orig, 10)

print("one hot label shape", y_train.shape)

### モデルのロード
01で保存したFashion-MNISTモデルをロードします

In [ ]:
USER    = "username" # 自分の名前
BUCKET  = "mixi-ml-handson-2021"
VERSION = "001"

base_model = tf.keras.models.load_model("gs://{}/{}/{}".format(BUCKET, USER, VERSION))

# ベースモデルを一時保存しておく
_, base_model_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(base_model, base_model_file, include_optimizer=False)

### ベースモデルの精度確認
再度、ベースモデルの評価を確認してみます。

In [ ]:
base_model.evaluate(X_test, y_test, batch_size=32)

### 重みの確認

pruningとは、重みが小さいエッジを取り去って、パラメータを削減する手法になります。  
パラメータが少なくなれば、その分モデルのサイズは小さくなり、高速化されます。  
しかし、今回のモデルの重みに削減する余地はあるでしょうか。

実際に重みの値を確認してみましょう。

まず、再度モデルの構成を確認します。

In [ ]:
base_model.summary()

この中のうち、`conv2d`と`dense`が層を構成しています。  
これらの層の重みからヒストグラムを作成してみましょう。

In [ ]:
import matplotlib.pyplot as plt

def draw_weights_histgram(model, layers_index, bins=1000):
    ## <todo> ___を埋めて指定したindexのweightsを渡せるようにしましょう
    weight_list = model.layers[___].weights[0].numpy().flatten()
    plt.hist(weight_list, bins=bins)


In [ ]:
## <todo>引数 layers_indexの部分にconv2dまたはdense層のindexを入れて、それぞれの重みをplotしてみましょう
## ヒント: モデルの構成を参考にしてみてください
## weightsの総数が少ない場合は、binsの値を小さくしてplotしてみてください
draw_weights_histgram(base_model, layers_index=___)

だいたいどの層をplotしてみても、0.0付近に値が集中していたのではないでしょうか。  
0.0付近のweightは、消去しても精度に大きな影響を与えないはずなので、このモデルにはpruningする余地が十分あるといえそうです。

### pruningモデルを定義
公式の[Pruning in Keras example](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)を参考にpruningモデルを定義します。

In [ ]:
import tensorflow_model_optimization as tfmot

def compute_necessary_steps(batch_size, epochs):
    return np.ceil(X_train.shape[0] / batch_size).astype(np.int32) * epochs

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
bigin_step = 0
end_step = compute_necessary_steps(batch_size=32, epochs=5)

## <todo> Pruning in Keras exampleを参考に'pruning_shcedule'を定義してみましょう
## 最初に10%をpruning、最終的には70%をpruningする様にスケジューリングしてみてください
pruning_params = {

}

model_for_pruning = prune_low_magnitude(base_model, **pruning_params)

In [ ]:
model_for_pruning.compile(
    optimizer='adam',
    loss="categorical_crossentropy", 
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

### 学習
pruningモデルが定義できたので、再学習させます。

In [ ]:
%rm -rf ./pruning_logs

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir='pruning_logs'),
]
model_for_pruning.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.1, callbacks=callbacks)

### 評価
学習が終わったら、これまでと同じように評価してみましょう。

In [ ]:
model_for_pruning.evaluate(X_test, y_test, batch_size=32)

モデルの精度はベースモデルと比較してどうなっているでしょうか。  
ほとんど変わってなければ、精度に影響を与えずにpruningされていることになります。

### 可視化
01と同じように、学習結果をtensorboardで可視化してみます。

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir pruning_logs

学習の推移やshcedule通りにpruningされていったかなどを確認してみてください。

### pruningモデルを圧縮
pruningすることが出来たので、モデルの圧縮を行いましょう。

[公式](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras#create_3x_smaller_models_from_pruning)によると、圧縮を確認するには`tfmot.sparsity.keras.strip_pruning`と標準の圧縮アルゴリズムの適用（gzipなど）の両方が必要とのことなので、
その対応をしていきます。

In [ ]:
## <todo> 公式を参考に___を埋めてpruningしたmodelにstrip_pruningを適応しましょう
model_for_export = ___

# pruningしたモデルを一時保存
_, pruned_model_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_model_file, include_optimizer=False)


In [ ]:
# gzipを適応した後のsizeをkbで返す関数
def get_gzipped_model_size_kb(file):
    # Returns size of gzipped model, in bytes.
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    return int(os.path.getsize(zipped_file) / 1024)

準備ができたので、各モデルにおける圧縮の効果を確認してみましょう。

In [ ]:
print("base model size    : {} kb".format(get_gzipped_model_size_kb(base_model_file)))
print("pruned model size : {} kb".format(get_gzipped_model_size_kb(pruned_model_file)))

モデルが1/3ほどに圧縮されたことが確認できているでしょうか。